In [ ]:
#take llm's to interpret abbrevations and expand the query accordingly for better result
#advanced: detect if term is medical and then instead of llm's use a dictionary as it is more precise
#optionals:
    #detect if query is a question query and prompt LLM to answer question, then expand query with keywords
    #get query and analyze most important part through LLM, then append that part multiple times to query     

# LLM's interprete abbrevations

In [43]:
#imports
import openai
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

In [44]:
client = openai.OpenAI() #connect to OpenAI API

In [45]:
def ask_gpt(prompt, model="gpt-4", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

In [46]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

In [47]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

alter query

In [ ]:
#iterate through each query in pt_dataset.get_topics('query')
#for query decide whether this query contains an abbrevation or not
#if query contains abbrevation expand the query through writing out the abbrevation

In [48]:
querys = pt_dataset.get_topics('query')
querys.head(3)

,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [49]:
print(querys['query'][0])

retrieval system improving effectiveness


determine if query contains abbrevation using gpt

In [62]:
answers = dict()
for i in range(len(querys)):
    determine_abbrevation = f""" 
    You are an scientific expert especially in the domain of Information Retrieval. Your task is to detect whether
    a given query, which is given as a text below delimited by triple quotes, contains an abbrevation then answer with yes or not then answer with no.
    For example given a query 'What is crx' you should answer yes, since cxr is the abbrevation for the medical term
    'chest X-Ray'. However if the given query is 'What is Information Retrieval' you should answer no, since there is no
    abbrevation in the query.

    query: '''{querys['query'][i]}'''
    """
    answer = ask_gpt(prompt=determine_abbrevation) #check answer more carefully perhaps model will return not only {yes,no}
    #print(answer)
    qid = str(querys['qid'][i])
    if "yes" in answer.lower().strip():
        answers[qid] = True
    else:
        answers[qid] = False
print(answers)

{'1': False, '2': False, '3': False, '4': False, '5': False, '6': False, '7': False, '8': False, '9': False, '10': False, '11': True, '12': False, '13': False, '14': False, '15': False, '16': False, '17': False, '19': False, '20': False, '21': False, '22': False, '23': False, '24': False, '25': False, '26': False, '27': False, '28': False, '29': False, '30': False, '31': False, '32': False, '33': False, '34': False, '35': False, '36': False, '37': False, '38': False, '39': False, '40': False, '41': False, '42': False, '43': False, '44': False, '45': False, '46': False, '47': False, '48': False, '49': False, '50': False, '51': False, '52': False, '53': False, '54': False, '55': True, '56': False, '57': False, '58': False, '59': True, '60': False, '61': False, '62': False, '63': False, '64': False, '65': False, '66': False, '67': False, '68': False, '18': False}


Expand query which contains abbrevation

In [69]:
#could be more efficient if in answers text is saved only for qid where abbrevation=yes
for key in answers.keys():
    if bool(answers[key]):
        #find query
        for i in range(len(querys)):
            if querys['qid'][i] == key:
                query= querys['query'][i]
                print(query)
        #ask gpt to expand query
        expand = f""" 
        You are an scientific expert especially in the domain of Information Retrieval. Your are given a query, which is below
        delimited by triple quotes, which contains an abbrevation. Your task is to identify the abbrevation and write it, then
        concat the written out abbrevation with the original query and return this new query as string only.
        For example given a query 'What is crx' you should detect that the abbrevation is crx, since cxr is the abbrevation for the medical term
        'chest X-Ray', then your should concat the abbrevation 'chest X-Ray' with the originial query resulting in a new query 'chest X-Ray What is crx' which
        you should return. Another example, given the query 'Algorithms of nlp' you should detect that the abbrevation is nlp, since nlp is the abbrevation
        for the term 'natural language processing', then you should concat the abbrevation 'natural language processing' with the original query 'Algorithms of nlp' 
        resulting in a new query 'natural language processing Algorithms of nlp' which you should return.

        query: '''{query}'''
        """
        new_query = ask_gpt(prompt=expand)
        print(new_query)
        #overwrite old query

algorithm acceleration with nvidia cuda
The abbreviation in the query is 'cuda'. CUDA stands for 'Compute Unified Device Architecture'. Therefore, the new query after concatenating the written out abbreviation with the original query is 'Compute Unified Device Architecture algorithm acceleration with nvidia cuda'.
bm25
The abbreviation in the query is 'bm25', which stands for 'Best Matching 25'. Therefore, the new query after concatenating the written out abbreviation with the original query is 'Best Matching 25 bm25'.
what is ahp
The abbreviation in the query is 'ahp', which stands for 'Analytic Hierarchy Process'. Therefore, the new query after concatenating the written out abbreviation with the original query is 'Analytic Hierarchy Process what is ahp'.


In [ ]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")